In [1]:
#pip install requests
#pip install beautifulsoup4
#pip install requests
#pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import json
import time
import re

url ='https://en.wikipedia.org/w/index.php?search=heroes+of+1821+greece&title=Special%3ASearch&ns0=1'
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')
entries =soup.find_all('li', class_='mw-search-result mw-search-result-ns-0')
#print(entries)
data = []
temp_id = 1

for entry in entries: 
    title_element = entry.find('div', class_='mw-search-result-heading')
    date_element = entry.find('div', class_='mw-search-result-data')

    if title_element and date_element:
        first_title = title_element.text.strip()
        date_parts = re.split(r'[-\n]', date_element.text.strip())
        infos = date_parts[0].strip() if len(date_parts) > 1 else "N/A"
        date = date_parts[1].strip() if len(date_parts) > 1 else "N/A"
        
       
        link_element = title_element.find('a')
        
        if link_element and 'href' in link_element.attrs:
            content_url = 'https://en.wikipedia.org' + link_element['href']
            ID = temp_id
            temp_id = temp_id + 1
        else:
            print("No href attribute found")
            continue  

        content_html = requests.get(content_url)
        content_soup = BeautifulSoup(content_html.content, 'html.parser')
        content_element = content_soup.find('div', class_='mw-content-ltr mw-parser-output')
        
        content = content_element.text.strip() if content_element else "N/A"

        data.append([first_title, ID, infos, date, content])
        print(f'First_Title: {first_title}\n\n ID: {ID}n\\n Informations: {infos}\n\n Date: {date}\n\nContent: {content}\n{"-" * 50}')
    
    time.sleep(1)


json_filename = "results.json"
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=2)

print(f"Τα δεδομένα αποθηκεύτηκαν στο αρχείο: {json_filename}")


First_Title: Greek War of Independence

 ID: 1n\n Informations: 175 KB (21,181 words)

 Date: 12:33, 9 January 2025

Content: Greek rebellion against the Ottoman Empire (1821–1829)


Greek War of IndependencePart of the Revolutions during the 1820sTop left: The camp  at Phaliro. Top right: The burning of an Ottoman frigate by a Greek fire ship. Bottom right: The Battle of Navarino. Bottom left: Ibrahim Pasha of Egypt at the Third Siege of Missolonghi.Date21 February 1821 – 12 September 1829[2](8 years, 6 months and 3 weeks)LocationOttoman Greece (present-day Greece)Result
Greek victory[a]Territorialchanges
Independence of Greece

The Peloponnese, Saronic Islands, Cyclades, Sporades and Continental Greece ceded to the independent Greek state
Crete ceded to EgyptBelligerents
 Greek Revolutionaries


In detail:
 Klephts
 Armatoloi
 Filiki Eteria [nb 1]
 Sacred Band [nb 1]
 Messenian Senate [nb 1]
 Peloponnesian Senate [nb 2]
 Senate of Western Continental Greece [nb 2]
 Areopagus of Easte

In [ ]:
Ο παραπάνω κώδικας με την χρήση της Beautiful soup λαμβάνει τον κώδικα html από την σελίδα της wikipedia. Με την entry.find() βρίσκει μεσα στον Html 
κώδικα τον τίτλο, ημερομηνία, πληροφορίες, Links Κλπ. Αν υπάρχει τίτλος και ημερομηνία, για τον τίτλο κρατάει το κείμενο χωρίς τα whitespaces.
η ημερομηνία λαμβάνεται με την χρήση της re.split από την βιβλιοθήκη re. H split χωρίζει ένα string σε substrings με βάση την εσωτερική παράμετρο που έχει
δηλωθεί. Τα δεδομένα του info και του date είναι το πρώτο στοιχείο και το δεύτερο στοιχείο αντίστοιχα του date_parts χωρίς του λευκούς χαρακτήρες
(πχ space) Μετά κατευθύνεται στο εκάστοτε άρθρο μέσω του Link απο το href και συλλέγει το περιεχόμενο του με την επανάληψη του beautiful soup στην 
συγκεκριμένη σελίδα. Για κάθε άρθρο εκχωρείται ένα μοναδικό ID, το οποίο μόλις εκχωρηθεί η τιμή, αυξάνεται κατά ένα. Τέλος, όλα τα δεδομένα εισάγονται 
στην λίστα data με την εντολή append και αυτά τέλος αποθηκεύονται σε αρχείο με όνομα results.json.

In [2]:
#pip install pandas
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

#nltk.download('punkt_tab')
#pip install nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

def preprocess_text(text):
    #tokenization
    words = word_tokenize(str(text))  # Μετατρέπουμε σε string για να αντιμετωπίσουμε ακέραιους
    
    processed_words = []
    
    for word in words:
        #check if the word is a number
        if word.isdigit():
            processed_words.append(str(word))  # Αποθηκεύουμε τον ακέραιο ως string
        else:
            #remove punctuation( Αφαίρεση σημείων στίξης)
            word = word.strip(string.punctuation)
            
            #stemming(κοβω τις καταληξεις)
            stemmer = PorterStemmer()
            word = stemmer.stem(word)
            
            #lemmatization(απλοποιω τις λεξεις)
            lemmatizer = WordNetLemmatizer()
            word = lemmatizer.lemmatize(word)
            
            #remove stop words(σβήνω λέξεις χωρίς νόημα)
            stop_words = set(stopwords.words('english'))
            if word.lower() not in stop_words:
                processed_words.append(word.lower())
    
    return ' '.join(processed_words)

#διαβάζω το αρχείο results.json
df = pd.read_json(r'D:\OneDrive - University of West Attica\σχολη ΠΑΔΑ\Εξάμηνα\Z Εξάμηνο\ΑΝΑΚΤΗΣΗ ΠΛΗΡ\2d_lab_try\results.json')


#εφαρμόζω την προεπεξεργασία σε κάθε κελί του πίνακα
for column in df.columns:
    df[column] = df[column].apply(preprocess_text)

#ορίζει τα νέα ονόματα για τις στήλες
new_columns = {
    0: 'First_Title',
    1: 'ID',
    2: 'Informations',
    3: 'Date',
    4: 'Content',
}

#μετονομάζει τις στήλες χρησιμοποιώντας το λεξικό που ορίσαμε
df = df.rename(columns=new_columns)

# Εφαρμογή του regex στη στήλη 'Content', αν υπάρχει
if 'Content' in df.columns:  # Ελέγχει αν υπάρχει στήλη 'text'
    df['Content'] = df['Content'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', str(x)))

# Αποθηκεύω το νέο αρχείο processed.json
json_filename = "processed.json"
with open(json_filename, 'w', encoding='utf-8') as json_file:
    # Χρησιμοποιούμε τη μέθοδο to_json της pandas
    df.to_json(json_file, force_ascii=False, indent=2, orient="records")

print("Η επεξεργασία ολοκληρώθηκε και τα δεδομένα αποθηκεύτηκαν στο αρχείο processed.json.")


Η επεξεργασία ολοκληρώθηκε και τα δεδομένα αποθηκεύτηκαν στο αρχείο processed.json.


In [ ]:
Με την χρήση της βιβλιοθήκης Pandas και τη συνάρτηση της βιβλιοθήκης read_json διαβάζεται το έγγραφο results.json, το οποίο περιλαμβάνει μέσα τα 
περιεχόμενα, τίτλο κλπ από το προηγούμενο ερώτημα και καταχωρείται στην μεταβλητή df. Για κάθε στήλη που περιλαμβάνεται στην μεταβλητή εκτελείται η
συνάρτηση που έχει δημιουργηθεί με όνομα Preprocess_text. Η συνάρτηση preprocess_text λαμβάνει την παράμετρο text την μετατρέπει σε token με την χρήση
της βιβλιοθήκης nltk.tokenize και φτιάχνει μία άδεια λίστα processed_words. Για κάθε λέξη μέσα στην λίστα words ελέγχει αν είναι αριθμός, αν είναι την 
μετατρέπει σε string και την βάζει στην λίστα της processed_words. Αλλιώς, αφαιρεί απο την λέξη σημεία στίξης και επόμενο βήμα γίνεται το stemming. Στο 
stemming κόβονται οι καταλήξεις απο τις λέξεις. Το lemmatization απλοποιεί τις λέξεις και τέλος για την συνάρτηση σβήνει τα stop words και τα προσθέτει 
ως πεζά στο processed_words και επιστρέφεται η Processed_words με κενά ανάμεσα στις λέξεις στο main πρόγραμμα. Στην συνέχεια ορίζονται οι νέες στήλες
και μετονομάζουμε το df με τις στήλες που δημιουργήσαμε απο το new_columns. Τέλος, ελέγχει αν υπάρχει η στήλη text και εφαρμόζει regex στη στήλη Content
αν υπάρχει και όλα αποθηκεύονται σε ένα νέο Json αρχείο με όνομα processed.

In [3]:
import json

# Διαβάζω το αρχείο JSON
with open(r'D:\OneDrive - University of West Attica\σχολη ΠΑΔΑ\Εξάμηνα\Z Εξάμηνο\ΑΝΑΚΤΗΣΗ ΠΛΗΡ\2d_lab_try\processed.json', 'r') as f:
    documents = json.load(f)  # Φορτώνω τα δεδομένα

# Λίστα που θα περιέχει όλους τους όρους
all_terms = []

# Αντίστροφο ευρετήριο
inverted_index = {}
boolean_inverted_index = {}
temp = {}

# Αποθήκευση όλων των επεξεργασμένων δεδομένων
# all_data = []

# Επεξεργασία κάθε εγγράφου
# for doc_id, document in enumerate(documents):
    # Δημιουργούμε νέο λεξικό για το κάθε έγγραφο
  #  processed_data = {}
   # keys_to_extract = ['First_Title', 'ID', 'Content']
    
    # Εξάγουμε τα επιθυμητά πεδία
    # for key in keys_to_extract:
      #  if key in document:  # Ελέγχουμε αν το πεδίο υπάρχει
       #     processed_data[key] = document[key]
    
    # Προσθέτουμε το επεξεργασμένο έγγραφο στη λίστα
    #all_data.append(processed_data)

# Επεξεργασία κάθε εγγράφου
for document in documents: # for document in all_data:
    doc_id = document.get('ID')
    for field, field_value in document.items():
        if field == "ID": # ερώτηση
            continue
        if isinstance(field_value, str):  # Ελέγχω αν το πεδίο είναι string
            terms = field_value.split()
            all_terms.extend(terms)  # Προσθήκη όρων στη λίστα για όλα τα έγγραφα

            # Δημιουργία του ανεστραμμένου ευρετηρίου
            for term in terms:
                if term in inverted_index:
                    inverted_index[term].append(doc_id)
                else:
                    inverted_index[term] = [doc_id]
    
for term in inverted_index:
    unique_ids = set(inverted_index[term])  # Αφαιρούμε τα διπλότυπα
    sorted_ids = sorted(unique_ids, key=int)  # Ταξινομούμε αριθμητικά
    inverted_index[term] = sorted_ids  # Μετατροπή σε λίστα και ενημέρωση του inverted_index
    
for term in inverted_index:
    temp = inverted_index[term]
    if term not in boolean_inverted_index:
        boolean_inverted_index[term] = []
    for index in range(20):
        if index < len(temp) and temp[index] == str(index + 1):
            boolean_inverted_index[term].append(str(1))
        else:
            boolean_inverted_index[term].append(str(0))
            
    
    
# Αποθήκευση του ανεστραμμένου ευρετηρίου σε ένα αρχείο JSON
with open('inverted_index.json', 'w') as json_file:
    json.dump(inverted_index, json_file)
    
# Αποθήκευση του ανεστραμμένου ευρετηρίου σε ένα αρχείο JSON
with open('boolean_inverted_index.json', 'w') as json_file:
    json.dump(boolean_inverted_index, json_file)

# Δοκιμαστική εκτύπωση μερικών όρων
print(all_terms[:10])

['greek', 'war', 'independ', '177', 'kb', '21,354', 'word', '05:03', '10', 'novemb']


In [ ]:
Το πρόγραμμα ξεκινάει με το φόρτωμα του αρχείου processed.json σε μία λίστα documents. Για κάθε εγγραφή στα documents λαμβάνει το id αυτού του στοιχείου
και για κάθε πεδίο και την τιμή του πεδίου μέσα στα στοιχεία του document και αν είναι ID βγαίνει εκτός επανάληψης, ειδάλως ελέγχει αν το πεδίο είναι 
string. Οι όροι των στοιχείων είναι απο το πεδίο, μετά απο split και προστείθονται στην λίστα all_terms. Ακόμα, δημιουργείται το αντεστραμμένο ευρετήριο,
δηλαδή αν ο όρος υπάρχει μέσα στην λίστα inverted_index προσθέτεται το Id. Αλλιώς, εκχωρείται στην θέση του όρου το νέο id. Μέσα στον Inverted_index για
κάθε όρο αφαιρούμε τα διπλότυπα, τα ταξινομούμε αριθμητικά με την εντολή sorted και μετατρέπουμε την λίστα και ενημέρωση του inverted_index. 
δημιουργούμε τον boolean inverted index για κάθε άρθρο που υπάρχει στην αναζήτηση μας με σταθερή τιμή 20. Αν το Index είναι μικρότερο του Inverted_index
[term] και ο όρος στην θέση index είνα ίσος με το index+1 τότε προστείθεται η τιμή 1 ως string, αλλιώς 0, για να σηματοδοτήσει οτι δεν εμφανίζεται. 
Τέλος, αποθηκεύονται οι Inverted index και boolean inverted index ως json σε αντίστοιχα αρχεία inverted_index.json και boolean_inverted_index.json.


In [5]:
import json
import math
from collections import Counter

def user_interface():
    print("Καλώς ήρθατε στην μηχανή αναζήτησης για τους ήρωες του 1821!")
    num_query = 0
    ap_score = [] 
    while True:
        query = input("Εισαγάγετε το ερώτημά σας ή γράψτε exit για έξοδο: ").strip().lower()
        
        if query == 'exit':
            if num_query !=0 :
                mAP = calculate_mAP(num_query,ap_score)
                print(f"Το mAP score είναι {mAP}")
            print("Ευχαριστούμε που χρησιμοποιήσατε τη μηχανη αναζήτησης μας.")
            break

        algorithm_choice = input("Επιλέξτε αλγόριθμο ανάκτησης:\n1) Boolean Retrieval\n2) Vector Space Model (VSM)\n3) Probabilistic Retrieval Models(OKAPI BM25)\nΕπιλογή: ").strip()

        if algorithm_choice == '1':
            # Αυξάνεται ο αριθμός των queries
            num_query = num_query + 1
            retrieved_documents, relevant_documents =boolean_retrieval(query)
            ap_score.append(calculate_ap(retrieved_documents,relevant_documents))
            print(f"Εκτέλεση Boolean Retrieval για το ερώτημα: {query} ")
        elif  algorithm_choice == '2':
            # Αυξάνεται ο αριθμός των queries
            num_query = num_query + 1
            print(f"Εκτέλεση Vector Space Model για το ερώτημα: {query}")
            ranked_documents,retrieved_documents, relevant_documents = vector_space_model(query)
            ap_score.append(calculate_ap(retrieved_documents,relevant_documents))
            print(f"Αποτελέσματα Vector Space Model: {ranked_documents}")
            
        elif algorithm_choice == '3':
            # Αυξάνεται ο αριθμός των queries
            num_query = num_query + 1
            ranked_documents,retrieved_documents, relevant_documents = probabilistic_retrieval(query)
            ap_score.append(calculate_ap(retrieved_documents,relevant_documents))
        else:
            print("Μη έγκυρη επιλογή αλγορίθμου. Παρακαλώ εισάγετε έναν έγκυρο αριθμό.")

def boolean_retrieval(query):
    inverted_index =  load_inverted_index_for_boolean()
    processed = load_processed()
    
    retrieved_documents = []
    relevant_documents = []
    
    if inverted_index is None:
        print("Η λειτουργία Boolean Retrieval δεν είναι δυνατή λόγω απουσίας αρχείου inverted_index.json.")
        return
    if processed is None:
        print("Η λειτουργία Boolean Retrieval δεν είναι δυνατή λόγω απουσίας αρχείου inverted_index.json.")
        return
    result = []
    
    if "and" in query:
        query = query.replace('and', ' ')
        terms = query.split()
        #print(f"Εκτέλεση Boolean Retrieval για το ερώτημα: {terms}")
        #print (terms)
        result_and = "1"*20
 
        for term in terms:
            if term.lower() in inverted_index:
                result.append(inverted_index[term.lower()])
 #           result1.update(processed[term.lower()])
            result_and = result_and and inverted_index[term]
        print(f"Το αποτέλεσμα από το and: {result_and}")
        
        
        for i, value in enumerate(result_and):
            if value == '1':
                relevant_documents.append(f"{i+1}")
                retrieved_documents.append(f"{i+1}")
            else:
               retrieved_documents.append(f"{i+1}") 
               
        relevant_documents = set(relevant_documents)
        retrieved_documents = set(retrieved_documents)
        precision, recall, F1_score = evaluate_precision_recall(retrieved_documents, relevant_documents)
        
        print(f"Το precision είναι {precision}")
        print(f"Το recall είναι {recall}")
        print(f"Το F1_score είναι {F1_score}")
        
        return retrieved_documents, relevant_documents
    elif "or" in query:
        query = query.replace('or', ' ')
        terms = query.split()
        print(f"Εκτέλεση Boolean Retrieval για το ερώτημα: {terms}")
        print (terms)
            #result = []
    #    result1 = set()
        result_or = "1"*20
     
        for term in terms:
            if term.lower() in inverted_index:
                result.append(inverted_index[term.lower()])
     #           result1.update(processed[term.lower()])
            result_or = result_or or inverted_index[term]
        result_or = list(result_or)
        print(f"result of or: {result_or}")
        
        for i, value in enumerate(result_or):
            if value == '1':
                relevant_documents.append(f"{i+1}")
                retrieved_documents.append(f"{i+1}")
            else:
               retrieved_documents.append(f"{i+1}") 
               
        relevant_documents = set(relevant_documents)
        retrieved_documents = set(retrieved_documents)
        precision, recall, F1_score = evaluate_precision_recall(retrieved_documents, relevant_documents)
        
        print(f"Το precision είναι {precision}")
        print(f"Το recall είναι {recall}")
        print(f"Το F1_score είναι {F1_score}")
        
        return retrieved_documents, relevant_documents
        
    elif "not" in query:
       query = query.replace('not', ' ')
       terms = query.split()
       print(f"Εκτέλεση Boolean Retrieval για το ερώτημα: {terms}")
    
       result_not = "1" * 20  # Ξεκινάμε με όλα τα έγγραφα (όλα "1")

       for term in terms:
            if term.lower() in inverted_index:
                result.append(inverted_index[term.lower()])
 #           result1.update(processed[term.lower()])
            result_not = result_not and inverted_index[term]
       print(f"Αποτέλεσμα NOT !: {result_not}")
       size_of_list = len(result_not)
       for i in range(size_of_list):
            if result_not[i] == '1':
                result_not[i] = '0'
            else:
                result_not[i] = '1'

       print(f"Αποτέλεσμα NOT: {result_not}")
       
       for i, value in enumerate(result_not):
           if value == '1':
               relevant_documents.append(f"{i+1}")
               retrieved_documents.append(f"{i+1}")
           else:
              retrieved_documents.append(f"{i+1}") 
              
       relevant_documents = set(relevant_documents)
       retrieved_documents = set(retrieved_documents)
       precision, recall, F1_score = evaluate_precision_recall(retrieved_documents, relevant_documents)
       
       print(f"Το precision είναι {precision}")
       print(f"Το recall είναι {recall}")
       print(f"Το F1_score είναι {F1_score}")
       
       return query,retrieved_documents, relevant_documents
    
def vector_space_model(query):
    inverted_index = load_inverted_index()

    if inverted_index is None:
        print("Η λειτουργία Vector Space Model δεν είναι δυνατή λόγω απουσίας αρχείου inverted_index.json.")
        return

    terms = query.split()
    print(f"Εκτέλεση Vector Space Model για το ερώτημα: {terms}")

    document_vectors = {}
    query_vector = Counter(terms)
    
    retrieved_documents = []
    relevant_documents = []

    for term, query_term_freq in query_vector.items():
        lower_term = term.lower()
        if lower_term in inverted_index:
            idf = math.log10(len(inverted_index) / len(inverted_index[lower_term]))

            for doc_id in inverted_index[lower_term]:
                doc_term_freq = inverted_index[lower_term].count(doc_id)
                tf_idf = doc_term_freq * idf

                if doc_id in document_vectors:
                    document_vectors[doc_id] += tf_idf * query_term_freq
                else:
                    document_vectors[doc_id] = tf_idf * query_term_freq

    # Ταξινόμηση των εγγράφων βάσει του cosine similarity
    ranked_documents = sorted(document_vectors.items(), key=lambda x: x[1], reverse=True)
    
    for ID, score in ranked_documents:
        if score >= 5.0:
            relevant_documents.append(f"{ID}")
            retrieved_documents.append(f"{ID}")
        else:
            retrieved_documents.append(f"{ID}")
    
    precision, recall, F1_score = evaluate_precision_recall(retrieved_documents, relevant_documents)
    
    print(f"Το precision είναι {precision}")
    print(f"Το recall είναι {recall}")
    print(f"Το F1_score είναι {F1_score}")        
    
    return ranked_documents,retrieved_documents, relevant_documents

def probabilistic_retrieval(query):
    processed = load_processed()

    if processed is None:
        print("Η λειτουργία Probabilistic Retrieval δεν είναι δυνατή λόγω απουσίας αρχείου processed.json.")
        return

    # Σταθερές για το BM25
    k1 = 1.5
    b = 0.75
    N = len(processed)  # Συνολικός Αριθμός των εγγράφων
    
    # Υπολογισμός του συνολικού αριθμού λέξεων στο "Content"
    total_words = 0
    for doc in processed:
        if "Content" in doc:  # Ελέγχει αν υπάρχει το πεδίο "Content"
            content = doc["Content"]
            total_words += len(content.split())  # Διαχωρισμός σε λέξεις και μέτρηση
    
    # Υπολογισμός του avgdl (μέσος αριθμός λέξεων ανά έγγραφο)
    if N > 0:
        avgdl = total_words / N
    else:
        N = 0

    # Διαχωρισμός του ερωτήματος σε όρους
    terms = query.split()
    scores = {}
    
    retrieved_documents = []
    relevant_documents = []

    for doc in processed:
        if "Content" not in doc:
            continue  # Παράβλεψη εγγράφων χωρίς πεδίο "Content"

        content = doc["Content"]
        doc_length = len(content.split())
        doc_term_counter = Counter(content.lower().split())  # Καταμέτρηση συχνότητας όρων στο έγγραφο
        score = 0

        for term in terms:
            term = term.lower()  # Μετατροπή του όρου σε πεζά για σύγκριση
            ft = doc_term_counter.get(term, 0)  # Συχνότητα του όρου στο έγγραφο

            # Υπολογισμός Document Frequency (DF)
            
            df = 0 
            for d in processed:
                if "Content" in d and term in d["Content"].lower():
                    df = df + 1

            if df == 0:
                continue  # Παράβλεψη όρων που δεν εμφανίζονται σε κανένα έγγραφο

            # Υπολογισμός IDF
            idf = math.log((N - df + 0.5) / (df + 0.5) + 1)

            # Υπολογισμός TF
            tf = (ft * (k1 + 1)) / (ft + k1 * (1 - b + b * (doc_length / avgdl)))

            # Υπολογισμός BM25 score
            score += idf * tf

        # Αποθήκευση του σκορ για το συγκεκριμένο έγγραφο
        scores[doc["ID"]] = score  # Υποθέτουμε ότι τα έγγραφα έχουν πεδίο "ID"
        if score != 0.0:
            retrieved_documents.append(doc["ID"])
        if score >= 0.5:
            relevant_documents.append(doc["ID"])

    # Ταξινόμηση των εγγράφων κατά αύξουσα σειρά του ID
    ranked_documents = sorted(scores.items(), key=lambda x: float(x[1]), reverse= True)

    # Εκτύπωση των αποτελεσμάτων
    print(f"Αποτελέσματα Probabilistic Retrieval για το ερώτημα '{query}':")
    for doc_id, score in ranked_documents:
        print(f"Έγγραφο ID: {doc_id}, Σκορ: {score}")
    print(f"Αποτελέσματα από retrieved_documents: {retrieved_documents}")
    print(f"Αποτελέσματα από relevant_documents: {relevant_documents}")
    precision, recall, F1_score = evaluate_precision_recall(retrieved_documents, relevant_documents)
    
    print(f"Το precision είναι {precision}")
    print(f"Το recall είναι {recall}")
    print(f"Το F1_score είναι {F1_score}")
    
    return ranked_documents,retrieved_documents, relevant_documents

#φορτώνω τα στοιχεία του boolean_inverted_index.json 
def load_inverted_index_for_boolean(file_path='boolean_inverted_index.json'):
    
    try:
        with open(file_path,'r') as json_file:
            inverted_index = json.load(json_file)
        return inverted_index
    except FileNotFoundError:
        print(f"Το αρχείο {file_path} δεν βρέθηκε.")
        return None
          
#φορτώνω τα στοιχεία του inverted_index.json 
def load_processed(file_path='processed.json'):
    try:
        with open(file_path,'r') as processed_file:
            processed = json.load(processed_file)
        return processed
    except FileNotFoundError:
        print(f"Το αρχείο {file_path} δεν βρέθηκε.")
        return None
    

#φορτώνω τα στοιχεία του inverted_index.json 
def load_inverted_index(file_path='inverted_index.json'):
    
    try:
        with open(file_path,'r') as json_file:
            inverted_index = json.load(json_file)
        return inverted_index
    except FileNotFoundError:
        print(f"Το αρχείο {file_path} δεν βρέθηκε.")
        return None

def evaluate_precision_recall(retrieved_documents, relevant_documents):
    retrieved_set = set(retrieved_documents)
    relevant_set = set(relevant_documents)
    
    true_positives = len(retrieved_set & relevant_set)
    precision = true_positives / len(retrieved_set) if retrieved_set else 0
    recall = true_positives / len(relevant_set) if relevant_set else 0
    
    F1_score = 2*((precision*recall)/(precision + recall))
    
    return precision, recall, F1_score

# Υπολογίζεται το MAP 

def calculate_mAP(queries,ap_score):
    return sum(ap_score)/queries
        
def calculate_ap(retrieved, relevant):
    precision_at_k = []
    num_relevant = len(relevant)
    num_correct = 0

    for i, item in enumerate(retrieved):
        if item in relevant:  # Ελέγχουμε αν το στοιχείο είναι σχετικό
            num_correct += 1
            precision_at_k.append(num_correct / (i + 1))  # Υπολογισμός precision@k

    # Επιστροφή του μέσου όρου των τιμών ακρίβειας
    if num_relevant == 0:
        return 0  # Αν δεν υπάρχουν σχετικά στοιχεία
    return sum(precision_at_k) / num_relevant
if __name__ == "__main__":
    user_interface()


Καλώς ήρθατε στην μηχανή αναζήτησης για τους ήρωες του 1821!


Εισαγάγετε το ερώτημά σας ή γράψτε exit για έξοδο:  greek and war
Επιλέξτε αλγόριθμο ανάκτησης:
1) Boolean Retrieval
2) Vector Space Model (VSM)
3) Probabilistic Retrieval Models(OKAPI BM25)
Επιλογή:  1


Το αποτέλεσμα από το and: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
Το precision είναι 1.0
Το recall είναι 1.0
Το F1_score είναι 1.0
Εκτέλεση Boolean Retrieval για το ερώτημα: greek and war 


Εισαγάγετε το ερώτημά σας ή γράψτε exit για έξοδο:  athanasio diako
Επιλέξτε αλγόριθμο ανάκτησης:
1) Boolean Retrieval
2) Vector Space Model (VSM)
3) Probabilistic Retrieval Models(OKAPI BM25)
Επιλογή:  3


Αποτελέσματα Probabilistic Retrieval για το ερώτημα 'athanasio diako':
Έγγραφο ID: 8, Σκορ: 0.9187066356849081
Έγγραφο ID: 6, Σκορ: 0.7994114283962427
Έγγραφο ID: 19, Σκορ: 0.7717832574555678
Έγγραφο ID: 17, Σκορ: 0.7706585025779226
Έγγραφο ID: 20, Σκορ: 0.7603262176122831
Έγγραφο ID: 9, Σκορ: 0.749807408439648
Έγγραφο ID: 15, Σκορ: 0.7325671316794723
Έγγραφο ID: 3, Σκορ: 0.7325505720702697
Έγγραφο ID: 13, Σκορ: 0.7059206432303248
Έγγραφο ID: 16, Σκορ: 0.6647171265164293
Έγγραφο ID: 7, Σκορ: 0.6440383858380371
Έγγραφο ID: 1, Σκορ: 0.6255048691870705
Έγγραφο ID: 10, Σκορ: 0.4661203948187655
Έγγραφο ID: 14, Σκορ: 0.35492881988926367
Έγγραφο ID: 11, Σκορ: 0.1225116369769036
Έγγραφο ID: 4, Σκορ: 0.10453941053475263
Έγγραφο ID: 5, Σκορ: 0.09915885974480507
Έγγραφο ID: 18, Σκορ: 0.0877999825147256
Έγγραφο ID: 12, Σκορ: 0.047414806223556115
Έγγραφο ID: 2, Σκορ: 0.0
Αποτελέσματα από retrieved_documents: ['1', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '1

Εισαγάγετε το ερώτημά σας ή γράψτε exit για έξοδο:  greek war
Επιλέξτε αλγόριθμο ανάκτησης:
1) Boolean Retrieval
2) Vector Space Model (VSM)
3) Probabilistic Retrieval Models(OKAPI BM25)
Επιλογή:  2


Εκτέλεση Vector Space Model για το ερώτημα: greek war
Εκτέλεση Vector Space Model για το ερώτημα: ['greek', 'war']
Το precision είναι 1.0
Το recall είναι 1.0
Το F1_score είναι 1.0
Αποτελέσματα Vector Space Model: [('1', 5.6285617984930365), ('2', 5.6285617984930365), ('3', 5.6285617984930365), ('4', 5.6285617984930365), ('5', 5.6285617984930365), ('6', 5.6285617984930365), ('7', 5.6285617984930365), ('8', 5.6285617984930365), ('9', 5.6285617984930365), ('10', 5.6285617984930365), ('11', 5.6285617984930365), ('12', 5.6285617984930365), ('13', 5.6285617984930365), ('14', 5.6285617984930365), ('15', 5.6285617984930365), ('16', 5.6285617984930365), ('17', 5.6285617984930365), ('18', 5.6285617984930365), ('19', 5.6285617984930365), ('20', 5.6285617984930365)]


Εισαγάγετε το ερώτημά σας ή γράψτε exit για έξοδο:  exit


Το mAP score είναι 0.898705676227606
Ευχαριστούμε που χρησιμοποιήσατε τη μηχανη αναζήτησης μας.


In [ ]:
Ο κώδικας καλεί την user_interface(), η συνάρτηση αυτή υλοποιεί το περιβάλλον του χρήστη, με το οποίο θα ζητάει το ερώτημα ή την λέξη "exit" για έξοδο
και η απάντηση του αποθηκεύεται σε μία μεταβλητή query.Το πρόγραμμα άμα βρεί την λέξη "exit", ως είσοδος τερματίζεται η εφαρμογή και εμφανίζει εάν και
εφόσον υπάρχει το mAP. Αν Ο χρήστης δώσει οποιαδήποτε άλλη απάντηση, ερωτάται να διαλέξει έναν απο τους τρείς αλγορίθμους, είτε τον Boolean Retrieval, 
είτε Vector Space Model (VSM), είτε Probabilistic Retrieval Models(OKAPI BM25). Για κάθε ένα αυξάνεται ο μετρητής των ερωτημάτων και καλείται η συνάρτηση
του εκάστοτε αλγορίθμου. Για την συνάρτηση του boolean_retrieval() λαμβάνεται ως παράμετρος το query. Στην συνέχεια φορτώνονται οι boolean_inverted_index
.json και processed.json μέσω συναρτήσεων, αν δεν υπάρχουν τα αρχεία εμφανίζεται κατάλληλο μήνυμα. Το πρόγραμμα ψάχνει μέσα στο ερώτημα του χρήστη, την
ύπαρξη and, or ή not. Αν ο χρήστης ψάχνει με την 'and', με την χρήση της εντολής replace το 'and', γίνεται το κενό. Οι όροι διασπόνται και εισάγουμε στη
μεταβλητή result_and 20 φορές το string "1". Ο λόγος που επιλέχθηκε το 1, 20 φορές, ήταν διότι υπάρχουν 20 άρθρα στην ιστοσελίδα, σχετικά με τους ήρωες
του 1821. Οτιδήποτε τιμή με την and μας κάνει το οτιδήποτε, οπότε είναι μία ασφαλής αρχικοποίηση για την υλοποίηση του and. Για κάθε όρο γίνεται σε η 
πράξη του and και παράγεται το αποτέλεσμα, το οποίο προστείθεται στο result_and. Μόλις έχει υπολογιστεί η τιμή της result_and, για κάθε θέση και τιμή 
εκχωρείται στα σχετικά και ανακτημένα έγγραφα. Επίσης με την συνάρτηση evaluate_precision_recall() υπολογίζεται  η ανάκληση και η ακρίβεια, όπως και το 
F1 score. Η συνάρτηση evaluate_precision_recall εφαρμόζει τον τύπο για τα true positives, βρίσκοντας το Precision και το recall και υλοποιεί την 
F1_score με την χρήση του τύπου. Τέλος επιστρέφει τις 3 τιμές πίσω στο πρόγραμμα, εκτυπώνονται και επιστρέφει τις τιμές. Με την ίδια λογική υλοποιείται 
και η or. Σχετικά με την not, αντιστρέφει απο την result_and τους άσσους σε μηδενικά και τα μηδενικά σε άσσους και συνεχίζει την διαδικασία όπως 
περιγράφτηκε παραπάνω.
Το VSM ή Vector Space Model κρατάει το πλήθος των όρων και για κάθε όρο και συχνότητα του όρου μέσα απο τη λίστα query_vector. Μετέπειτα, χρειάζεται η
εύρεση του idf, του οποίου ο τύπος είναι ο λογάριθμος με βάση 10 το Length του inverted_index προς το μήκος του inverted index στην θέση lower_term. Για
κάθε id μέσα στο Inverted index στην στήλη lower term βρίσκουμε την συχνότητα, την πολλαπλασιάζουμε με το idf και υπολογήστηκε το tf_idf. Ακόμα,
προστείθεται στο document_vectors στην θέση του doc_id. Στην συνέχεια ταξινομείται και αν το score είναι μεγαλύτερο ή ίσο του 5 τοτε προσθέτεται στο 
σχετικά και ανακτηθέντα έγγραφα με το id. Αλλιώς εισάγεται στα ανακτηθέντα.
Ο τελευταίος αλγόριθμος είναι ο probabilistic_retrieval. O Okapi BM25 χρησιμοποιεί τα δεδομένα απο το processed.json και με τις σταθερές k1,b και ο 
συνολικός αριθμός των εγγράφων. Ο αλγόριθμος μετά απο την λίστα doc ψάχνει την στήλη Content, μετατρέποντας τα σε πεζά και βρίσκει την συχνότητα του 
εγγράφου. Μετά υπολογίζει το Document Frequency (DF), το IDF, το Term Frequency και τέλος αποθηκεύει το σκόρ για το συγκεκριμένο έγγραφο στην λίστα
scores. Αν το σκόρ του εγγράφου είναι μεγαλύτερο του 0.5 τότε εκχωρείται στην λίστα σχετικών εγγράφων και αν είναι διάφορο του μηδενός εισάγεται και 
στην λίστα ανακτηθέντων. Τέλος ταξινομούνται τα έγγραφα, υπολογίζονται οι ανακλήσεις, ακρίβειες και το F1_score και επιστρέφουν πίσω. Το πρόγραμμα 
ολοκληρώνεται με τον υπολογισμό του mAP μέσω αντίστοιχων συναρτήσεων